# Data cleaning and pre-processing
### In this notebook, we clean and pre-process the raw dataset provided by Iowa State University. 

In [1]:
# Import data handling libraries
import pandas as pd
import numpy as np

# Import cleaner functions
import cleaner

#read in the raw dataset
df = pd.read_csv('../Data/data_set_10_17.csv')

df.sample(5)

,STUDENT,MAJOR_CURR,MAJOR_COLL,CLSFN_YR,ENTRY_SEM_CD,ENTRY_CCYY,LEVEL,SEM_CCYY,SEM_CD,OFFER_DEPT_ABRVN,...,DEG_DATEDB,MAJOR_CURR.1,DEG_DESCR,LAST_RGST_TERM,CLSFN_YR.1,MAJOR_CURR.2,MAJOR_COLL.1,Workday Enrolled in Fall 2024,Workday Enrolled in Fall 2024 Class Standing,Workday Enrolled in Fall 2024 Primary Program of Study
21544,9237,COM S,S,2,F,2022,U,2023,F,MATH,...,NaN,NaN,NaN,S24,3.0,COM S,S,717738037.0,Junior,"Computer Science, B.S."
22019,9438,P CS,S,1,F,2016,U,2016,F,MATH,...,5/9/2020,COM S,B.S. LIBERAL ARTS & SCIENCES ...,S20,4.0,COM S,S,NaN,NaN,NaN
21473,9195,COM S,S,2,F,2013,U,2015,F,MATH,...,NaN,NaN,NaN,F16,3.0,COM S,S,NaN,NaN,NaN
23461,10080,COM S,S,4,1,2014,U,2016,S,MATH,...,5/5/2018,MATH,B.S. LIBERAL ARTS & SCIENCES ...,F22,6.0,MATH,S,NaN,NaN,NaN
14074,6019,STAT,S,2,F,2018,U,2018,F,MATH,...,11/28/2020,STAT,B.S. LIBERAL ARTS & SCIENCES ...,F20,5.0,LAS S,S,NaN,NaN,NaN


Before we start with any data cleaning, let's first get a quick look at the columns and statistics in the dataset.

In [2]:
cleaner.print_column_summary(df)
cleaner.gradestats(df)

Column Summary:
Column Name               Description                                       
---------------------------------------------------------------------------
STUDENT                   student identifier (integer)                      
MAJOR_CURR                major at time of taking the course                
MAJOR_COLL                College of liberal arts for all courses           
CLSFN_YR                  Classification of the year (Freshman, Sophomore, etc.) when course was taken
ENTRY_SEM_CD              Semester of enrollment ('F' --> Fall, 'S' --> Spring, '1' --> Summer)
ENTRY_CCYY                Year of enrollment                                
LEVEL                     Irrelevant, to be dropped                         
SEM_CCYY                  Year in which course was taken                    
SEM_CD                    Semester in which course was taken                
OFFER_DEPT_ABRVN          Same for all rows (Math department)               
CRSE            

Each student comes with a unique integer identifier. The rows correspond to each course taken by the student. In the final dataset, we will have only one row per student, and the columns will correspond to courses. So, we begin our cleaning with:

1. Drop unnecessary columns at the outset.

In [3]:
df = cleaner.drop_initial_cols(df)
df.columns

Index(['STUDENT', 'ENTRY_CCYY', 'ENTRY_SEM_CD', 'SEM_CCYY', 'SEM_CD',
       'SEM_CCYY.1', 'SEM_CD.1', 'DEG_CD', 'GRADE_CATGORY', 'CRSE',
       'CRSE_TITLE'],
      dtype='object')

2. Next, we format the date/semester columns for entry, course, and graduation. We will combine each pair into a single unique identifier (float), named `ENT_SEM`, `CRSE_SEM`, and `GRAD_SEM`.
    - For enrollment, we encode Spring, Summer and Fall as $0.0$, $0.4$, and $0.6$, respectively.   
    - For course and graduation, we encode Spring, Summer and Fall as $0.4$, $0.6$, and $1.0$, respectively.
    - The new columns are computed by the formula (year mod 2000) + (sem value)
    - Thus, if a student enrolled in Fall 2012, and took a particular course in Spring 2014, then `ENT_SEM`$ = 12.6$ and `CRSE_SEM`$ = 14.4$. 
    - Similarly, if they graduated in Spring 2016, then `GRAD_SEM`$ = 16.4$.

In [4]:
df = cleaner.format_dates(df)
df[['ENT_SEM','CRSE_SEM','GRAD_SEM']].sample(5)

,ENT_SEM,CRSE_SEM,GRAD_SEM
11482,17.6,18.4,NaN
3639,16.6,20.0,23.0
23562,18.6,21.0,22.4
7129,18.6,19.0,22.4
17378,15.6,16.0,20.0


3. Next, we drop students who enrolled before 2008 (because data is shaky), and also those who enrolled in Spring 2021 or later (since we cannot yet determine if these student will have graduated in 4 years or not).

In [5]:
df = cleaner.remove_by_ENT_SEM(df)

After removing students who entered before 2008:
Rows: 30219
Students: 13013
Grads: 7578
Grad rate: 58.23

After dropping students who enrolled in Spring 2021 or later:
Rows: 24855
Students: 10012
Grads: 7385
Grad rate: 73.76


4. Next, we drop all students who received a graduate degree. (Note, we completely drop the student even though they may have also received a bachelors degree. This is because it is not clear whether a particular course was taken towards the bachelors degree or graduate degree).

In [6]:
df = cleaner.remove_graduate_degrees(df)

After dropping students who took classes towards a graduate degree
Rows: 22362
Students: 9689
Grads: 7062
Grad rate: 72.89


5. Next, we drop all rows corresponding to courses taken after graduation. 
    
    We also re-format `CRSE_SEM` and `GRAD_SEM` to contain the timestamp relative to `ENT_SEM`, and we multiply by $2.5$ so that fall and spring count as 1 semester and Summer counts as half a semester (so a full year counts as $2.5$ semesters). 
    
    Our final target variable will be $1$ if the student "graduates in 4 years"-- for us, this means `GRAD_SEM` is at most $9.5$, which is the typical case when a student enrolls in Fall and graduates in Spring 4 years later.
    
    Accordingly, we won't consider courses taken after $9.5$ years, so we drop all rows where `CRSE_SEM` is greater than $9.5$.

In [7]:
df = cleaner.remove_by_CRSE_SEM(df)

After dropping courses taken after 9.5 semesters:
Rows: 21068
Students: 9401
Grads: 6821
Grad rate: 72.56

After removing courses taken after graduation:
Rows: 21059
Students: 9398
Grads: 6818
Grad rate: 72.55


6. We remove any duplicate rows at this point. (These exist because some of the original rows were repeated for each degree earned by the student.)

In [8]:
df = cleaner.remove_duplicates(df)

Number of duplicate rows: 1184
After dropping duplicates:
Rows: 19875
Students: 9398
Grads: 6818
Grad rate: 72.55


7. Next, we format the course numbers and titles, combine together similar courses, and drop courses that are no longer offered, or which have an overall low enrollment (<50).

In [9]:
df = cleaner.format_and_drop_CRSE(df)

After formatting courses and titles, and removing courses with low enrollment:
Rows: 18802
Students: 9181
Grads: 6647
Grad rate: 72.4


Before continuing with the cleaning, let's pause to make a csv with the remaining courses and their enrollment counts. We will store the courses in a list for the next step of cleaning.

In [10]:
df, courses = cleaner.make_crse_counts_csv(df)

Left with 28 courses: ['165', '166', '265', '143', '140', '104', '150', '207', '201', '317', '267', '105', '301', '266', '414', '160', '101', '25', '492', '341', '385', '435', '397', '436', '495', '314', '500', '304']


8. Next, we format the `GRADE_CATGORY` column, using the value $1$ for 'ABOVE C-' and 'Satisfactory/Pass', and $-1$ for 'C- OR BELOW'.

    Then, for each `x` in `courses` (list of all `CRSE` values appearing), we create a column which contains the grade ($\pm 1$) times the course semester (a float between $0.4$ and $3.8$). Thus, the values in `df[x]` will range from $-3.8$ to $3.8$.

In [11]:
df = cleaner.make_CRSE_columns(df,courses)

9. Finally, we remove the unnecessary columns and aggregate the dataset by student, taking the value in each column that is largest in absolute value (that is, we keep only the data of the last time a particular course was taken by a student). Finally, we add a column `Y` which equals 1 if `GRAD_SEM` is at most $3.8$, and $0$ otherwise. 

In [12]:
final_cols = ['STUDENT','ENT_SEM','GRAD_SEM'] + courses
df = cleaner.agg_and_add_Y(df,final_cols)

After aggregating the dataset by student and adding the target variable:
Rows: 9181
Students 9181
Grads: 4496
Grad rate: 48.97


10. We're done! So we now save to csv.

In [13]:
#save the final dataset
df.to_csv('../Data/dataset_cleaned_11_30.csv', index=False)

At this point, the cleaning and pre-processing is done. We conclude this notebook by making dictionaries of the math major courses and general math courses.

In [14]:
crse_dict = \
{'101': 'ORIENTATION IN MATH',
 '104': 'INTRO TO PROBABILTY',
 '105': 'INTRO TO MATH IDEAS',
 '140': 'COLLEGE ALGEBRA',
 '143': 'PREP FOR CALCULUS',
 '150': 'DISC MATH BUS&SOC S',
 '160': 'SURVEY OF CALCULUS',
 '165': 'CALCULUS I',
 '166': 'CALCULUS II',
 '201': 'INTRO TO PROOFS',
 '207': 'MATRCES&LINEAR ALGB',
 '25': 'HIGH SCHOOL ALGEBRA',
 '265': 'CALCULUS III',
 '266': 'ELEM DIFF EQUATNS',
 '267': 'DIFF EQ & TRANSFMS',
 '301': 'ABSTRACT ALGEBRA I',
 '304': 'COMBINATORICS',
 '314': 'GRAPH THEORY',
 '317': 'THRY LINEAR ALGEBRA',
 '341': 'INTR THY PROBAB&S I',
 '385': 'INTR PART DIFF EQUA',
 '397': 'TEACH SEC SCHL MATH',
 '414': 'ANALYSIS I',
 '435': 'GEOMETRY I',
 '436': 'GEOMETRY II',
 '492': 'UNDERGRAD SEMINAR',
 '495': 'INDEPENDENT STUDY',
 '500': 'GRADUATE MATH'}

gen_dict = \
{'104': 'INTRO TO PROBABILTY',
 '105': 'INTRO TO MATH IDEAS',
 '140': 'COLLEGE ALGEBRA',
 '143': 'PREP FOR CALCULUS',
 '150': 'DISC MATH BUS&SOC S',
 '160': 'SURVEY OF CALCULUS',
 '165': 'CALCULUS I',
 '166': 'CALCULUS II',
 '207': 'MATRCES&LINEAR ALGB',
 '25': 'HIGH SCHOOL ALGEBRA',
 '265': 'CALCULUS III',
 '266': 'ELEM DIFF EQUATNS',
 '267': 'DIFF EQ & TRANSFMS'}

math_dict = \
{'101': 'ORIENTATION IN MATH',
 '201': 'INTRO TO PROOFS',
 '301': 'ABSTRACT ALGEBRA I',
 '304': 'COMBINATORICS',
 '314': 'GRAPH THEORY',
 '317': 'THRY LINEAR ALGEBRA',
 '341': 'INTR THY PROBAB&S I',
 '385': 'INTR PART DIFF EQUA',
 '397': 'TEACH SEC SCHL MATH',
 '414': 'ANALYSIS I',
 '435': 'GEOMETRY I',
 '436': 'GEOMETRY II',
 '492': 'UNDERGRAD SEMINAR',
 '495': 'INDEPENDENT STUDY',
 '500': 'GRADUATE MATH'}